### Librerias

In [1]:
import pandas_gbq
import plotly.express as px
import plotly.graph_objects as go
from google.oauth2 import service_account

### Funciones

In [2]:
# ubicacion del archivo json
credentials = service_account.Credentials.from_service_account_file('bigquery.json')
#
def excecute_query(query_string):
    df = pandas_gbq.read_gbq(query_string, project_id='bigquery-345316',
                                credentials=credentials,use_bqstorage_api=True)
    print(f"consultando sql a bigquery {df.size}")
    return df

### Ejecucion

In [3]:
dataset_bogota = excecute_query("""SELECT population, longitude_centroid, latitude_centroid, geog
                            FROM `bigquery-public-data.worldpop.population_grid_1km`
                            WHERE country_name='Colombia'
                            AND (latitude_centroid>4.36 AND latitude_centroid<4.832)
                            AND (longitude_centroid<-73.947 AND longitude_centroid>-74.252)
                            AND last_updated ='2019-01-01'
                            """)
dataset_bogota.head()

Downloading: 100%|██████████| 2109/2109 [00:01<00:00, 1457.42rows/s]

consultando sql a bigquery 8436


,population,longitude_centroid,latitude_centroid,geog
0,21563.138672,-74.188750,4.603750,"POLYGON((-74.1929163549 4.59958351631003, -74...."
1,20844.281250,-74.088750,4.678750,"POLYGON((-74.0929163553 4.67458351601003, -74...."
2,71.569290,-74.097083,4.362084,"POLYGON((-74.1012496886 4.35791685061002, -74...."
3,21012.031250,-74.163750,4.545417,"POLYGON((-74.167916355 4.54125018321002, -74.1..."
4,175.369873,-73.955416,4.712084,"POLYGON((-73.9595830225 4.70791684921003, -73...."


In [6]:
fig = px.scatter_mapbox(dataset_bogota, lat="latitude_centroid", lon="longitude_centroid",
                        color='population', size='population', size_max=9, zoom=9.6,
                        center={"lat": 4.6580945, "lon": -74.1028763}, color_continuous_scale='Jet')
fig.update_layout(mapbox_style="open-street-map",margin={"r":0,"t":0,"l":0,"b":0},)
fig.update_layout(template='plotly_dark')
fig.show()

In [5]:
dataset_colombia = excecute_query("""SELECT population, longitude_centroid, latitude_centroid, geog
                            FROM `bigquery-public-data.worldpop.population_grid_1km`
                            WHERE country_name='Colombia' AND last_updated ='2019-01-01'
                            """)
dataset_colombia.head()

Downloading: 100%|██████████| 1330965/1330965 [00:22<00:00, 60093.70rows/s]


consultando sql a bigquery 5323860


,population,longitude_centroid,latitude_centroid,geog
0,2.709957,-72.372083,4.720417,"POLYGON((-72.3762496955 4.71625018251002, -72...."
1,0.506016,-72.997083,0.012084,"POLYGON((-73.001249693 0.00791686801002165, -7..."
2,0.339326,-69.988750,3.853750,"POLYGON((-69.9929163717 3.84958351931002, -69...."
3,31.102722,-76.113750,4.170417,"POLYGON((-76.1179163472 4.16625018471002, -76...."
4,332.690216,-75.680416,6.245417,"POLYGON((-75.6845830156 6.24125017641003, -75...."


In [6]:
dataset_colombia.memory_usage(deep=True)

Index                       128
population             10647720
longitude_centroid     10647720
latitude_centroid      10647720
geog                  307826586
dtype: int64

In [ ]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(dataset_colombia.columns),align='left'),
    cells=dict(values=dataset_colombia.values.T,align='left'))
])
#fig.update_layout(template='plotly_dark')
fig.show()

### BIG QUERIES

In [17]:
dataset_world = excecute_query("""SELECT country_name, population, longitude_centroid, latitude_centroid
                                FROM `bigquery-public-data.worldpop.population_grid_1km`
                                WHERE last_updated ='2019-01-01' AND country_name IS NOT NULL
                                """)
dataset_world.head()

Downloading: 100%|██████████| 217886205/217886205 [02:15<00:00, 1603200.49rows/s]


consultando sql a bigquery 871544820


,country_name,population,longitude_centroid,latitude_centroid
0,Burkina Faso,19.661880,-2.622083,14.012083
1,Guinea,61.022854,-9.313750,11.712083
2,Burkina Faso,17.866865,-2.372083,11.628750
3,Burkina Faso,34.894630,-3.447083,11.395417
4,Burkina Faso,15.998444,-2.305417,11.812083


In [24]:
fig = px.scatter_mapbox(dataset_world[dataset_world['country_name']=='Andorra'],
                        lat="latitude_centroid", lon="longitude_centroid",text='country_name',
                        color='population', size='population', size_max=30, zoom=9.6,
                        color_continuous_scale='Jet')
fig.update_layout(mapbox_style="open-street-map",margin={"r":0,"t":0,"l":0,"b":0},)
fig.update_layout(template='plotly_dark')
fig.show()

In [19]:
# Extraer los ids de los paises con mas poblacion
id_max_population = dataset_world.groupby('country_name')['population'].idxmax()
id_max_population

country_name
Afghanistan                8519258
Albania                   16778370
Algeria                  111114991
American Samoa            19309005
Andorra                    4112577
                           ...    
Virgin Islands (U.S.)     15402510
Wallis and Futuna          6700552
Yemen                     25182807
Zambia                    13645170
Zimbabwe                  13975271
Name: population, Length: 232, dtype: int64

In [20]:
dataset_world[dataset_world['country_name']=='Mexico'].sort_values('population',ascending=False)
#dataset_world.loc[6354178]

,country_name,population,longitude_centroid,latitude_centroid
104395125,Mexico,447304.218750,-100.172083,25.895417
104345440,Mexico,339059.531250,-100.163750,25.903750
104962769,Mexico,207720.718750,-100.163750,25.895417
103755836,Mexico,145209.515625,-100.180416,25.887083
104346603,Mexico,132186.562500,-100.172083,25.887083
...,...,...,...,...
103497964,Mexico,0.000000,-107.122083,30.120417
104351919,Mexico,0.000000,-105.355416,27.712083
105153278,Mexico,0.000000,-105.072083,29.737083
104257153,Mexico,0.000000,-103.663750,24.728750


In [22]:
fig = px.scatter_mapbox(dataset_world.loc[id_max_population],
                        lat="latitude_centroid", lon="longitude_centroid",text='country_name',
                        color='population', size='population', size_max=15, zoom=1,
                        color_continuous_scale='Jet')
fig.update_layout(mapbox_style="open-street-map",margin={"r":0,"t":0,"l":0,"b":0},)
fig.update_layout(template='plotly_dark')
fig.show()